In [1]:
import numpy as np
import pandas as pd

In [2]:
data=pd.read_csv('diabetes.csv')
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
data.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [ ]:
#X=data[['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
      # 'BMI', 'DiabetesPedigreeFunction', 'Age']].values
#print('shape of X is ',X.shape)


#y=data['Outcome'].values.reshape(-1,1)
#print('shape of y',y.shape)

In [ ]:
#X_train =X[0:761]
#X_test=X[761:]

#print(X_train.shape)
#y_train=y[0:761].reshape(-1,1)
#y=y.reshape(-1,1)
#print(y_train.shape)
#print(X_test.shape)

In [4]:
mu1 = -1
mu2 = 3
sig1 = 0.5
sig2 = 1
N = 100
np.random.seed(10)
x11=np.random.randn(N,1)*sig1 + mu1
x12=np.random.randn(N,1)*sig1 + mu1+3
x21=np.random.randn(N,1)*sig2 + mu2
x22=np.random.randn(N,1)*sig2 + mu2+3
c = np.vstack((np.zeros((N,1)), np.ones((N,1))))
x1 = np.hstack((x11,x12))
x2 = np.hstack((x21,x22))

X = np.hstack( (np.vstack( (x1,x2) ),c) )
np.random.shuffle(X)
dataset = pd.DataFrame(data=X, columns=['x','y','c'])

In [5]:
dataset.head()

,x,y,c
0,-0.334207,2.058738,0.0
1,3.264395,5.636814,1.0
2,-0.669884,1.856185,0.0
3,1.995284,4.484924,1.0
4,3.588041,4.739172,1.0


In [6]:
X=dataset[['x','y']].values
y=dataset['c'].values

X_train=X[:190]

X_test=X[190:]
y_true=y[190:]
y_train=y[:190].reshape(-1,1)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)


(190, 2)
(190, 1)
(10, 2)


In [68]:
class LogisticRegresion:
    def __init__(self, lr=0.01, max_iteration=10):
        self.lr=lr
        self.max_iteration=max_iteration
        
    #hypothesis : logistic sigmoid function
    def sigmoid(self,X,theta):
        #s=1/(1 + np.exp(-1*(X@theta)))
        #print('sigmoid',s)
        z = X@theta
        #print('z', z.shape)
        return 1/(1 + np.exp(-z))
    
    # compute the grandient  
    def get_gradientB(self,X,y,theta):
        #return (1/len(X))*np.sum(( self.sigmoid(X,theta) - y)*X)
        h   = self.sigmoid(X,theta)
        return np.dot(X.T, (h - y))
    
    def get_gradientS(self,X,y,theta):
        return (1/len(X))*np.sum(( self.sigmoid(X,theta) - y)*X)
        #h   = self.sigmoid(X,theta)
        #return np.dot(X.T, (h - y))
    
    #compute the loss
    def loss(self,X,y,theta):
        return   (-y*np.log(self.sigmoid(X,theta))  - (1-y)*np.log(1- self.sigmoid(X,theta))).mean()
#         return   np.sum(((-1)*y*np.log(self.sigmoid(X,theta))  - (1-y)*np.log(1- self.sigmoid(X,theta))))
    
    
    def fit_BGD(self,X,y):
        
        #make a copy of our data
        X_copy=X.copy()
        
         #add the intercept column
        intercept=np.ones((X_copy.shape[0],1))
        X_copy=np.concatenate((intercept,X_copy),axis=1)
        #print('shape of the whole x',X_copy.shape)
        
        #initialise the weight
        self.theta=np.zeros((X_copy.shape[1],1))
        #print('shape of theta',theta.shape)
        
        for i in range(self.max_iteration):
            #compute the gradient
            grad = self.get_gradientB(X_copy, y, self.theta)#.reshape(-1,1)
            #update the weight
            self.theta = self.theta - self.lr*grad
            #print('my update',theta)
            #print the value of the loss
            print('the loss function is ',self.loss(X_copy,y,self.theta))
            
            
            
    def fit_SGD(self,X,y):
        
        #make a copy of our data
        X_copy=X.copy()
        
         #add the intercept column
        intercept=np.ones((X_copy.shape[0],1))
        X_copy=np.concatenate((intercept,X_copy),axis=1)
        #print('shape of the whole x',X_copy.shape)
        
        #initialise the weight
        self.theta=np.zeros((X_copy.shape[1],1))
        #print('shape of theta',theta.shape)
        
        for i in range(self.max_iteration):
            random_vector=np.random.permutation(X_copy.shape[0])
            for j in random_vector:
                
                #compute the gradient
                grad = self.get_gradientS(X_copy[j], y[j], self.theta)#.reshape(-1,1)
                #update the weight
                self.theta = self.theta - self.lr*grad.reshape(-1,1)
                #print('my update',theta)
                #print the value of the loss
                print('the loss function is ',self.loss(X_copy[j],y[j],self.theta))

    def predict(self,X):
        X_copy=X.copy()
        
        print('shape of theta',self.theta.shape)
        intercept=np.ones((X_copy.shape[0],1))
        print('shape of x copy' ,X_copy.shape)
        print('shape of intercept' ,intercept.shape)
        X_copy=np.concatenate((intercept, X_copy), axis=1)
        print('shape',X_copy.shape)
        
        return self.sigmoid(X_copy,self.theta)

In [71]:
logistic=LogisticRegresion(lr=0.01,max_iteration=10)

In [77]:
logistic.fit_BGD(X_train,y_train)

the loss function is  1.2130960121047334
the loss function is  0.0760864537794677
the loss function is  0.04641383592372804
the loss function is  0.03854041950505316
the loss function is  0.03605987179184948
the loss function is  0.03471206996295768
the loss function is  0.03358986882195909
the loss function is  0.03256720942936033
the loss function is  0.03162232240807866
the loss function is  0.03074520384136152


In [73]:
logistic.fit_SGD(X_train,y_train)

the loss function is  0.5947153934302694
the loss function is  0.5699710667106251
the loss function is  0.725062764083486
the loss function is  0.4862650741256107
the loss function is  0.5130451391603877
the loss function is  0.758917879617918
the loss function is  0.47268817592645296
the loss function is  0.40282355725200386
the loss function is  0.3858546218032287
the loss function is  0.2769787452533128
the loss function is  0.7972297112737179
the loss function is  0.8067092584216645
the loss function is  0.8182568783402117
the loss function is  0.721535188453181
the loss function is  0.7215405896928055
the loss function is  0.39267449545536465
the loss function is  0.3651992128689557
the loss function is  0.2886969124350541
the loss function is  0.2920821626879536
the loss function is  0.8290043194119299
the loss function is  0.2555156650236543
the loss function is  0.25047187742694593
the loss function is  0.8490258081832786
the loss function is  0.8713431320855228
the loss functi

In [78]:
prediction = logistic.predict(X_test)

shape of theta (3, 1)
shape of x copy (10, 2)
shape of intercept (10, 1)
shape (10, 3)


Compute the accuracy 

In [79]:
y_hat = (prediction > 0.5).astype(int)

In [80]:
np.sum(y_hat == y_true.reshape(-1,1)) / len(y_true)

1.0

In [ ]:
X_train.shape

In [ ]:
y_train.squeeze()